In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from joblib import load,dump

In [2]:
df= pd.read_csv('data/Hotel_review_feature_engineering.csv')
df.head()

FileNotFoundError: [Errno 2] File data/Hotel_review_feature_engineering.csv does not exist: 'data/Hotel_review_feature_engineering.csv'

In [ ]:
class Review_recommender():
    def __init__(self,data=df):
        self.data = data
        self.clean_df = self.data.groupby('Hotel_Name').agg({'Negative_Review':', '.join,'Positive_Review':', '.join}).reset_index()
        self.clean_df['Combined_Review'] = self.clean_df['Positive_Review'].astype(str) + self.clean_df['Negative_Review'].astype(str)
        self.clean_df["Combined_Review"] = self.clean_df["Combined_Review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))
        self.clean_df['Combined_Review'] = self.clean_df['Combined_Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
        stop = stopwords.words('english')
        self.clean_df['Combined_Review'] = self.clean_df['Combined_Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
        tfv = TfidfVectorizer(min_df=3, max_features =None,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3))
        self.tfv_matrix = tfv.fit_transform(self.clean_df['Combined_Review'])
        self.clean_df.drop(['Combined_Review','Positive_Review','Negative_Review'],1,inplace=True)
        del tfv,self.data,stop
        self.cos_sim = linear_kernel(self.tfv_matrix, self.tfv_matrix)
        self.indices = pd.Series(self.clean_df.index, index=self.clean_df['Hotel_Name']).drop_duplicates()
        
    def recommend(self,index):
        index = self.indices[index]
        sim_scores = list(enumerate(self.cos_sim[index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        hotel_indices = [i[0] for i in sim_scores]
        for i in self.clean_df['Hotel_Name'].iloc[hotel_indices]:
            print(i)

In [ ]:
recommender = Review_recommender()
recommender.recommend('Hotel Arena')

In [ ]:
dump(recommender,'model/recommender_based_on_reviews.joblib')

In [ ]:
model = load('model/recommender_based_on_reviews.joblib')

In [ ]:
model.recommend('Hotel Pulitzer')